**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
!pip install scikit-video
!pip install opencv-python

     |████████████████████████████████| 2.3MB 25.4MB/s 


In [2]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

# A enlever
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = [15,7]


Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act selects the next action the agent will take at the state s. It implements an $\epsilon$-greedy strategy with $\epsilon = 0.1$ as default.
In training mode, the agent will either select the action with the best expected reward with a probability of $1- \epsilon$ or select a random action with probability $\epsilon$. This allows the agent to retrieve information about the environment and improve his estimations of the expected reward for all the states (with probability 1). It is a trade-off between exploration an exploitation.
In test mode, it can be better to always select the best action, as the goal is to maximize the reward.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=30 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

```position```: position is an $N \times N$ matrix that specify the rat's position (he is markes as a one) and the area where it's allowed to move (marked with 0), and not allowoed (marked with -1). It is  updated after each action.

```board``` : the variable board is an $N \times N$ matrix that keeps track of the reward of each cell of the  grid.
When the environment is reset, board yields a reward of 0.5 for the cells where there is cheese, and -1 for the poisonous cells. Each time the rat visits a cell, its reward is set to 0 (the rat eats what's in the cell) which is done the function act.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):

        a = np.random.randint(0, self.n_action, size=1)[0]

        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix='', verbose=True):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE

        state = env.reset()

        win = 0
        lose = 0

        while env.t < T :

          action = agent.learned_act(state)

          prev_state = state
          state, reward, game_over = env.act(action)

          if reward > 0:
            win = win + reward
          if reward < 0:
            lose = lose -reward

        #####
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose
        if verbose : 
          print("Win/lose count {}/{}. Average score ({})"
                .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent_rdm = RandomAgent()

test(agent_rdm,env,1,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.0/20.0. Average score (-11.0)
Final score: -11.0


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}


----

In the discounted inifinite case, the expression fot the $Q$ function is :

\begin{equation*}Q^\pi(s,a)=E_{(s',a')\sim p^{\pi}(.|s,a)}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]
\end{equation*}

Because of the Markov property, we can write : 
\begin{equation*}
= r(s,a) + \gamma \sum_{s'}\sum_{a'}p^{\pi}(s',a'|s,a)[\sum_{t=1}^{\infty}\gamma^{t-1}r(s_{t},a_{t})|s_{1}=s',a_{1}=a'] \> 
\end{equation*}

\begin{equation*}
= r(s,a) + \gamma \sum_{s'}\sum_{a'}p^{\pi}(s',a'|s,a)[\sum_{t'=0}^{\infty}\gamma^{t'}r(s_{t'},a_{t'})|s_{0}=s',a_{0}=a'] \> 
\end{equation*}

\begin{equation*}
= r(s,a) + \gamma \sum_{s'}\sum_{a'}p^{\pi}(s',a'|s,a) Q^{\pi}(s',a')\> 
\end{equation*}

\begin{equation*}
= E_{(s',a')\sim p^{\pi}(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')])\> 
\end{equation*}



----

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

The optimal $Q$ function is defined as : 

\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

\begin{equation*}
Q^*(s,a)=Q^{\pi^{*}}(s,a) \> .
\end{equation*}

$$=E_{(s',a')\sim {\pi*}(.|s,a)}[r(s,a)+\gamma Q^{\pi^{*}}(s',\pi^*(s'))]$$


$$=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma \max_{a'} Q^{\pi^{*}}(s',a')]$$

----

Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

The optimale $Q$ function is the only function that satisfies the Bellmann equation. Threfore, we want to fin the $\theta$ so that the difference :
$$r(s,a)+\gamma\max_{a'}Q^{*}(s',a')-Q^{*}(s,a)$$
is minimal (ideally 0) on average with respect to $p^{\pi^*}$. 
If $Q$ is $\theta$-parametrized, then minimising $\mathcal{L}(\theta)$ will make $Q_\theta$ converge to $Q^*$, with :
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}



***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) == self.max_memory :
          self.memory = self.memory[1:]+[m]
        else :
          self.memory.append(m)

    def random_access(self):
        rdn_id = np.random.randint(len(self.memory))
        m = self.memory[rdn_id]
        return m

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix='',verbose=True,plot=False):
    # Number of won games
    score = 0
    loss = 0
    scores, wins, losses = [], [], []

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        scores.append(score)
        wins.append(win)
        losses.append(lose)

        if verbose : 
          print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
                .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

    if plot : 
      plt.plot(wins, label="wins")
      plt.plot(losses, label="losses")
      plt.xlabel("Epoch")
      plt.legend()
      plt.grid()


***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

***
***

We trained the model for 50 epochs and for different sets of parameters, and used the average score over a few epochs on the test phase to evaluate it.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size
        

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.expand_dims(s, 0)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):

            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_ 

            # We only take into account the states and actions where we have new information. 
            # In the others, the target is the input, and they do not impact the gradient, thus the update of theta.
            target_q[i] =self.model.predict(np.expand_dims(s_, axis = 0))
            
            if game_over_:
                target_q[i,a_] = r_
            else:
                target_q[i,a_] = r_ + self.discount * np.max(self.model.predict(np.expand_dims(n_s_, axis = 0)))

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Dense(50, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Dense(25, activation = "relu"))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [38]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, 50, prefix='fc_train', verbose=False)
test(agent,env,25,prefix='fc_test', verbose = False)

Final score: 5.14


#### Neural Structure

##### A small model - 3.95



In [0]:
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Dense(32, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Dense(16, activation = "relu"))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, 50, prefix='fc_train', verbose=False)
test(agent,env,10,prefix='fc_test', verbose = False)

Final score: 3.95


##### A medium model (1) - 6.65

In [0]:
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Dense(64, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Dense(32, activation = "relu"))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, 50, prefix='fc_train', verbose=False)
test(agent,env,10,prefix='fc_test', verbose = False)

Final score: 6.65


##### A medium model (2) - 5.9

In [0]:
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Dense(128, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Dense(32, activation = "relu"))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, 50, prefix='fc_train', verbose=False)
test(agent,env,10,prefix='fc_test', verbose = False)

Final score: 5.9


##### A bigger model : 5.8

In [0]:
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Dense(256, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Dense(128, activation = "relu"))
        model.add(Dense(32, activation = "relu"))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, 50, prefix='fc_train', verbose=False)
test(agent,env,10,prefix='fc_test', verbose = False)

Final score: 5.8


#### batch_size

In [0]:
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Dense(64, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Dense(32, activation = "relu"))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 16)
train(agent, env, 50, prefix='fc_train', verbose=False)
test(agent,env,10,prefix='fc_test', verbose = False)

Final score: 4.25


In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, 50, prefix='fc_train', verbose=False)
test(agent,env,10,prefix='fc_test', verbose = False)

Final score: 5.25


In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64)
train(agent, env, 50, prefix='fc_train', verbose=False)
test(agent,env,10,prefix='fc_test', verbose = False)

Final score: 9.25


#### learning rate

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64)
train(agent, env, 50, prefix='fc_train', verbose=False)
test(agent,env,10,prefix='fc_test', verbose = False)

Final score: 7.2


In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.3, epsilon = 0.1, memory_size=2000, batch_size = 64)
train(agent, env, 50, prefix='fc_train', verbose=False)
test(agent,env,10,prefix='fc_test', verbose = False)

Final score: 5.7


In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.5, epsilon = 0.1, memory_size=2000, batch_size = 64)
train(agent, env, 50, prefix='fc_train', verbose=False)
test(agent,env,10,prefix='fc_test', verbose = False)

Final score: 6.55


#### memory size

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=500, batch_size = 64)
train(agent, env, 50, prefix='fc_train', verbose=False)
test(agent,env,10,prefix='fc_test', verbose = False)

Final score: 3.85


In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=1000, batch_size = 64)
train(agent, env, 50, prefix='fc_train', verbose=False)
test(agent,env,10,prefix='fc_test', verbose = False)

Final score: 3.7


In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64)
train(agent, env, 50, prefix='fc_train', verbose=False)
test(agent,env,10,prefix='fc_test', verbose = False)

Final score: 4.3


In [16]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=3500, batch_size = 64)
train(agent, env, 50, prefix='fc_train', verbose=False)
test(agent,env,10,prefix='fc_test', verbose = False)

Final score: 6.5


***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(64,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(32,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=3500, batch_size = 64)
train(agent,env,50,prefix='cnn_train')

Epoch 000/050 | Loss 0.0043 | Win/lose count 5.5/4.0 (1.5)
Epoch 001/050 | Loss 0.0041 | Win/lose count 11.0/9.0 (2.0)
Epoch 002/050 | Loss 0.0064 | Win/lose count 5.5/2.0 (3.5)
Epoch 003/050 | Loss 0.0310 | Win/lose count 10.5/2.0 (8.5)
Epoch 004/050 | Loss 0.0022 | Win/lose count 4.0/0 (4.0)
Epoch 005/050 | Loss 0.0032 | Win/lose count 2.0/1.0 (1.0)
Epoch 006/050 | Loss 0.0015 | Win/lose count 3.0/0 (3.0)
Epoch 007/050 | Loss 0.0016 | Win/lose count 6.0/4.0 (2.0)
Epoch 008/050 | Loss 0.0020 | Win/lose count 11.5/4.0 (7.5)
Epoch 009/050 | Loss 0.0015 | Win/lose count 2.5/4.0 (-1.5)
Epoch 010/050 | Loss 0.0021 | Win/lose count 8.5/1.0 (7.5)
Epoch 011/050 | Loss 0.0020 | Win/lose count 4.5/0 (4.5)
Epoch 012/050 | Loss 0.0029 | Win/lose count 7.5/1.0 (6.5)
Epoch 013/050 | Loss 0.0028 | Win/lose count 6.5/2.0 (4.5)
Epoch 014/050 | Loss 0.0018 | Win/lose count 7.5/4.0 (3.5)
Epoch 015/050 | Loss 0.0017 | Win/lose count 12.0/1.0 (11.0)
Epoch 016/050 | Loss 0.0021 | Win/lose count 4.5/2.0 (2.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

#### Temperature : 0.3

In [25]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)

agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

print('Test of the CNN')
test(agent_cnn,env,50,prefix='cnn_test', verbose=False)
print('Test of the FC')
test(agent_fc,env,50,prefix='fc_test', verbose=False)

Test of the CNN
Final score: 8.71
Test of the FC
Final score: 5.44


####Température : 0.1

In [28]:
env = Environment(grid_size=size, max_time=T,temperature=0.1)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

print('Test of the CNN')
test(agent_cnn,env,50,prefix='cnn_test', verbose=False)
print('Test of the FC')
test(agent_fc,env,50,prefix='fc_test', verbose=False)

Test of the CNN
Final score: 2.79
Test of the FC
Final score: 1.81


####Température : 0.6

In [31]:
env = Environment(grid_size=size, max_time=T,temperature=0.6)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

print('Test of the CNN')
test(agent_cnn,env,50,prefix='cnn_test', verbose=False)
print('Test of the FC')
test(agent_fc,env,50,prefix='fc_test', verbose=False)

Test of the CNN
Final score: 21.45
Test of the FC
Final score: 14.51


Using a convolutionnal neural network for the gain prediction is much more efficient than fully connected layers. 

When the temperature increases, there are more cells that yields positive reward and the scores are improved for both models. , but the gap between CNN and FC networks widens.

In all cases, the agent often ends up alternating between a few cells and not explore a (sometimes big) part of the environment, even if these cells no longer yield any reward.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix='',eps_decay=0.1, verbose=True, plot=False):
    score = 0
    loss = 0
    scores, wins, losses = [], [], []

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        agent.set_epsilon(agent.epsilon*(1-eps_decay))

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        scores.append(score)
        wins.append(win)
        losses.append(lose)

        if verbose : 
          print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
                .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

    if plot : 
      plt.plot(wins, label="wins")
      plt.plot(losses, label="losses")
      plt.xlabel("Epoch")
      plt.legend()
      plt.grid()


class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1, malus_pos=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        
        #board with previously visited cells
        self.malus_position = np.zeros((grid_size,grid_size))
        self.malus_pos = malus_pos
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1

        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = self.malus_pos
        reward = reward + self.board[self.x, self.y]

        self.board[self.x, self.y] = 0
        
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [34]:
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, 50, prefix='cnn_train_explore', verbose=False)
test(agent,env,50,prefix='cnn_test_explore')

Win/lose count 25.0/1.0. Average score (24.0)
Win/lose count 20.5/2.0. Average score (21.25)
Win/lose count 12.0/0. Average score (18.166666666666668)
Win/lose count 9.5/2.0. Average score (15.5)
Win/lose count 19.5/3.0. Average score (15.7)
Win/lose count 23.0/2.0. Average score (16.583333333333332)
Win/lose count 12.0/0. Average score (15.928571428571429)
Win/lose count 19.0/1.0. Average score (16.1875)
Win/lose count 29.0/2.0. Average score (17.38888888888889)
Win/lose count 22.5/1.0. Average score (17.8)
Win/lose count 9.5/0. Average score (17.045454545454547)
Win/lose count 17.0/0. Average score (17.041666666666668)
Win/lose count 10.5/0. Average score (16.53846153846154)
Win/lose count 19.5/1.0. Average score (16.678571428571427)
Win/lose count 18.0/1.0. Average score (16.7)
Win/lose count 20.5/0. Average score (16.9375)
Win/lose count 4.5/0. Average score (16.205882352941178)
Win/lose count 22.5/3.0. Average score (16.38888888888889)
Win/lose count 21.0/1.0. Average score (16.57

The agent is already a lot better ! To ensure more exploration during the first phase of the training, which is why the initial value for  ϵ  is increased. The  ```eps_decay``` parameter will gradually decrease its value, switching the strategy to a more exploitation strategy.
We increase the number of training epochs, to allow the agent to finish its (longer) exploration phase and the method to stabilize.

In [40]:
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.6, memory_size=3500, batch_size = 64,n_state=3)
train_explore(agent, env, 200, prefix='cnn_train_explore', verbose=False)
test(agent,env,50,prefix='cnn_test_explore', verbose=False)

Final score: 15.67


In [48]:
HTML(display_videos("cnn_test_explore46.mp4"))

The results have clearly improved. The agent is now less likely to cycle between to cells, and recieves greater rewards overall.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***